In [ ]:
lulc_rst = '/home/gisuser/autocls/osmlc_lisbon_nomixpx.tif'

idxrst = {
    'xlsx'    : '/home/gisuser/autocls/inputs_sheet.xlsx',
    'sheet'   : 'lisbon',
    'dayscol' : 'days'
}

idx_rules = '/home/gisuser/glass/exp/autoclsosm/idxrules_v2.json'

fraster = '/home/gisuser/autocls/osmlc_lisbon_idxfilter_v2.tif'

In [ ]:
import os
from osgeo import gdal
import numpy as np
from glass.rd.rst import rst_to_array
from glass.prop.rst import get_nodata
from glass.pys.oss import lst_ff, fprop
from glass.wenv.grs import run_grass
from glass.pys.tm import now_as_str
from glass.rd import tbl_to_obj
from glass.rd.js import json_to_obj

In [ ]:
idxs = tbl_to_obj(idxrst['xlsx'], sheet=idxrst['sheet'])

idxs.set_index(idxrst['dayscol'], inplace=True)

idxs = idxs.to_dict(orient='index')

rules = json_to_obj(idx_rules)

_rules = {int(k) : rules[k] for k in rules}

ndays = len(list(idxs.keys()))

for c in _rules:
    for idx in _rules[c]:
        if not _rules[c][idx]: continue

        _rules[c][idx]['nimgs'] = ndays if _rules[c][idx]['nimgs'] == 'all' \
            else int(_rules[c][idx]['nimgs'])

In [ ]:
_rules

In [ ]:
# Check if outfolder exists
ws = os.path.dirname(fraster)
if not os.path.exists(ws):
    mkdir(ws, overwrite=None)
    
"""
Start GRASS GIS Session
"""

loc = f'loc_{now_as_str()}'
grsb = run_grass(
    ws, grassBIN='grass78', location=loc,
    srs=lulc_rst
    )

import grass.script.setup as gsetup
    
gsetup.init(grsb, ws, loc, 'PERMANENT')

In [ ]:
from glass.it.rst import rst_to_grs, grs_to_rst
from glass.rst.alg import grsrstcalc

In [ ]:
# Identificar classes existentes no raster com as classes de LULC

lulc_img = rst_to_array(lulc_rst)

ndval = get_nodata(lulc_rst)

lulc_class = [v for v in np.unique(lulc_img) if v != ndval]

In [ ]:
print(lulc_class)

In [ ]:
# Send all rasters to GRASS GIS

lulc_grs = rst_to_grs(lulc_rst, fprop(lulc_rst, 'fn'))

for d in idxs:
    for k in idxs[d]:
        idxs[d][k] = rst_to_grs(idxs[d][k], fprop(idxs[d][k], 'fn'))

In [ ]:
idxs

Para cada classe, índice e data, criar um ficheiro que indica se o pixel deve ser considerado para treino (1) ou não (0)

In [ ]:
mask_by_class_day_idx = {}

for cls in lulc_class:
    mask_by_class_day_idx[cls] = {}
    for d in idxs:
        mask_by_class_day_idx[cls][d] = {}
        for k in idxs[d]:
            if not _rules[cls][k]: continue
            
            form = (
                f"if({lulc_grs} == {str(cls)} && "
                f"{idxs[d][k]} >= {_rules[cls][k]['min']} && "
                f"{idxs[d][k]} <= {_rules[cls][k]['max']}, "
                "1, 0)"
            )
            
            outmask = grsrstcalc(form, f'mask_{cls}_{k}_{d}')
            
            mask_by_class_day_idx[cls][d][k] = outmask
            
            grs_to_rst(outmask, os.path.join(
                ws, loc,
                f'{outmask}.tif'
            ), rtype=int)

Para cada classe e índice, criar um ficheiro que indica se o pixel deve ser incluído no treino (1) ou não (0), tendo em conta as várias máscaras obtidas para essa classe e índice em cada uma das datas consideradas

In [ ]:
cls_masks = {}

for cls in mask_by_class_day_idx:
    cls_masks[cls] = {}
    
    for d in mask_by_class_day_idx[cls]:
        for idx in mask_by_class_day_idx[cls][d]:
            if idx not in cls_masks[cls]:
                cls_masks[cls][idx] = []
            
            cls_masks[cls][idx].append(mask_by_class_day_idx[cls][d][idx])

In [ ]:
for cls in cls_masks:
    for idx in cls_masks[cls]:
        cntrst = grsrst_to_calc(
            " + ".join(cls_masks[cls][idx]),
            f'clsidxcount_{cls}_{idx}', ascmd=True
        )
        
        cls_masks[cls][idx] = grsrst_to_calc(
            f"if({cntrst} == {_rules[cls][idx]['nimgs']}, 1, 0)",
            f'clsidxmask_{cls}_{idx}', ascmd=True
        )

Obter máscara binária final para cada classe

In [ ]:
for cls in cls_masks:
    ifr = [f"{cls_masks[cls][idx]} == 1" for idx in cls_masks[cls]]
    
    exp = f"if({' && '.join(ifr)}, 1, 0)"
    
    cls_masks[cls] = grstscalc(exp, f"clsmask_{cls}", ascmd=True)

Obter ficheiro que indica as classes a considerar no treino - neste raster, o valor 0 indica que o pixel não deve ser considerado no treino.

In [ ]:
ifs = []

for c in cls_masks:
    exp = (
        f"if({cls_masks[c]} == 1, "
        f"{str(c)}, "
        f"{'0' if not len(ifs) else ifs[len(ifs)-1]})"
    )
    
    ifs.append(exp)

rfnl = grsrstcalc(
    ifs[-1],
    fprop(fraster, 'fn'), ascmd=True
)

grs_to_rst(
    rfnl, fraster, as_cmd=True,
    rtype=int, nodata=0
)